# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [49]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [50]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [51]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # skipping the header
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#get total number of rows 
print(len(full_data_rows_list))
# uncomment to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [52]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Use Apache Cassandra to Model the Database Tables. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [53]:
# make a connection to a Cassandra instance the local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [54]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class':'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [55]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create tables to complete the following three tasks.

### Task1: Getting the artist and song title and song's length  in the music app history that was heard during sessionId = 338, and itemInSession = 4


#### For the query, sessionId and itemInSession are considered as composite key because it perfectly filters the data as per the query we are answering from the table.

#### Create a table

In [56]:
# Create table song_detail_session
query1 = "CREATE TABLE IF NOT EXISTS song_detail_session"
query1 = query1 + "(sessionId int, itemInSession int, artist text, length float, song text,PRIMARY KEY (sessionId, ItemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)

#### Insert data

In [57]:
# read data from the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
# insert data into table song_detail_session
    for line in csvreader:
        query = "INSERT INTO song_detail_session (sessionId, itemInSession, artist, length, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0],float(line[5]),line[9]))

#### Check the table created

In [58]:
# Use a SELECT statement to verify the data was entered into the table
query = "SELECT artist,length,song from song_detail_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist,row.length,row.song)    

Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


### TASK 2: Getting the artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### For the query, sessionId and userId and ItemInSession are considered as the composite key where ItemInsession acts as the clustering key. Because it perfectly filters the data and songs are sorted by itemInsession.

#### Create a table

In [59]:
# create table song_playlist_session
query2 = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query2 = query2 + """(sessionId int, itemInSession int, userid int, artist text, song text, firstName text, lastName text,\
                        PRIMARY KEY ((sessionId,userId), ItemInSession))"""
try:
    session.execute(query2)
except Exception as e:
    print(e)      

#### Insert data

In [60]:
# read data from the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
# insert the data into table song_playlist_session    
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (sessionId, itemInSession, userid, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]),int(line[10]),line[0],line[9],line[1],line[4]))


#### Check the table created

In [24]:
# Use a SELECT statement to verify the data was entered into the table
query = "SELECT artist,song,firstname,lastname from song_playlist_session WHERE sessionId = 182 AND userid =10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname,row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### TASK3: Getting user names (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### For the query song and userId are considered as the composite key. Because we need to deal with the situaton when multiple users with the same name listen to the same song, and userId can identify each user. 

#### Create a table

In [43]:
# create table song_user_name
query3 = "CREATE TABLE IF NOT EXISTS song_user_name"
query3 = query3 + "(song text,userId int, firstName text, lastName text, PRIMARY KEY (song,userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)      

                    

#### Insert data

In [44]:
# read data from the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
# insert the data into table song_user_name    
    for line in csvreader:
        query = "INSERT INTO song_user_name (song,userId,firstName, lastName)"
        query = query + "VALUES (%s, %s, %s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

#### Check the table created

In [23]:
# Use a SELECT statement to verify the data was entered into the table
query = "SELECT firstName, lastName from song_user_name WHERE song='All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname,row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [45]:
# Drop the table before closing out the sessions
query = "drop table song_detail_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [46]:
# Drop the table before closing out the sessions
query = "drop table song_user_name"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [47]:
# Drop the table before closing out the sessions
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [48]:
session.shutdown()
cluster.shutdown()